In [1]:
import zipfile
trnsfile = '/content/drive/MyDrive/Colab Notebooks/end2/week15/week15_task3/simpletransformers-master.zip'

In [2]:
#%cd '/content/drive/MyDrive/Colab Notebooks/end2/week15'

In [3]:
with zipfile.ZipFile(trnsfile,"r") as zip_ref:
    zip_ref.extractall("/content/task3")

In [4]:
import sys
sys.path.append('/content/task3/simpletransformers-master')
#sys.path.append('/drive/My Drive/Medium')

In [5]:
pawsdata = '/content/drive/MyDrive/Colab Notebooks/end2/week15/week15_task3/paws_data.zip'

In [6]:
with zipfile.ZipFile(pawsdata,"r") as zip_ref:
    zip_ref.extractall("/content/pawsdata")

In [7]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 14.2 MB/s 
     |████████████████████████████████| 124 kB 63.8 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 3.3 MB 45.5 MB/s 
     |████████████████████████████████| 264 kB 89.0 MB/s 
     |████████████████████████████████| 1.6 MB 60.5 MB/s 
     |████████████████████████████████| 8.0 MB 79.8 MB/s 
     |████████████████████████████████| 1.2 MB 54.4 MB/s 
     |████████████████████████████████| 2.6 MB 52.1 MB/s 
     |████████████████████████████████| 895 kB 52.1 MB/s 
     |████████████████████████████████| 636 kB 74.5 MB/s 
     |████████████████████████████████| 97 kB 8.3 MB/s 
     |████████████████████████████████| 133 kB 76.8 MB/s 
     |████████████████████████████████| 170 kB 88.6 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 243 kB 82.6 MB/s 
     |████████████████████████████████| 118 kB 91.9 MB/s 
     |██████████████

In [8]:
import warnings

import pandas as pd


def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [9]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [10]:
# Google Data
train_df = pd.read_csv("/content/pawsdata/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("/content/pawsdata/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"


In [11]:
train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

print(train_df)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


           prefix  ...                                        target_text
1      paraphrase  ...  The 1975 -- 76 season of the National Basketba...
3      paraphrase  ...  The results are high when comparable flow rate...
4      paraphrase  ...  It is the seat of the district of Zerendi in A...
5      paraphrase  ...  William Henry Harman was born in Waynesboro, V...
7      paraphrase  ...  Given a discrete set of probabilities formula ...
...           ...  ...                                                ...
49384  paraphrase  ...  The Romance language currently spoken in Galic...
49390  paraphrase  ...  It is necessary to note that k is a vector con...
49393  paraphrase  ...  Tim Tim Henman won 6 -- 2, 7 -- 6 against Yevg...
49395  paraphrase  ...  He was considered an active member of the Coun...
49397  paraphrase  ...  She was at Cork on 24 June, and arrived in the...

[21829 rows x 3 columns]


In [12]:
len(train_df) , len(eval_df)


(21829, 3539)

In [13]:
scaledowntrainfactor = 5
scaleddownlen = len(train_df)//scaledowntrainfactor

In [14]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-base", # "facebook/bart-large",
    args=model_args,
)

model.train_model(train_df[:int(scaleddownlen)], eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
"""os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )"""

INFO:filelock:Lock 139778297700432 acquired on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.8512cdf8592f538a7fd4b40eecaa096285410ec6494049568b3300922ab71165.lock


Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

INFO:filelock:Lock 139778297700432 released on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.8512cdf8592f538a7fd4b40eecaa096285410ec6494049568b3300922ab71165.lock
INFO:filelock:Lock 139778297700432 acquired on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

INFO:filelock:Lock 139778297700432 released on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock
INFO:filelock:Lock 139778317168400 acquired on /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock


...

  0%|          | 0/3539 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.7035525547606605}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-base model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/56 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


'os.makedirs("predictions", exist_ok=True)\n\nwith open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:\n    for i, text in enumerate(eval_df["input_text"].tolist()):\n        f.write(str(text) + "\n\n")\n\n        f.write("Truth:\n")\n        f.write(truth[i] + "\n\n")\n\n        f.write("Prediction:\n")\n        for pred in preds[i]:\n            f.write(str(pred) + "\n")\n        f.write(\n            "________________________________________________________________________________\n"\n        )'

In [15]:
import logging

from simpletransformers.seq2seq import Seq2SeqModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

model = Seq2SeqModel(
    encoder_decoder_type="bart", encoder_decoder_name="outputs"
)


while True:
    original = input("Enter text to paraphrase: ")
    to_predict = [original]

    preds = model.predict(to_predict)

    print("---------------------------------------------------------")
    print(original)

    print()
    print("Predictions >>>")
    for pred in preds[0]:
        print(pred)

    print("---------------------------------------------------------")
    print()

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
Her life spanned years of incredible change for women as they gained more rights than ever before.

Predictions >>>
Her life spanned years of incredible change for women as they gained more rights than ever before.
Her life spanned years of incredible change for women as they gained more rights than ever before.
Her life spanned years of incredible change for women as they gained more rights than ever before.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.

Predictions >>>
Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.
Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.
Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
Symptoms of influenza include fever and nasal congestion.

Predictions >>>
Sym symptoms of influenza include fever and nasal congestion.
Sym symptoms of influenza include fever and nasal congestion.
Sym symptoms of influenza include fever and nasal congestion.
---------------------------------------------------------



Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------------------------------------
He has tons of stuff to throw away. 

Predictions >>>
He has tons of stuff to throw away.
He has tons of stuff to throw away.
He has tons of stuff to throw away.
---------------------------------------------------------



KeyboardInterrupt: ignored